In [1]:
import pandas as pd
import os
import numpy as np

import Utils as ut
import CortesAlignmentFile as ca
import mySampler as ms
 
from sklearn.model_selection import StratifiedShuffleSplit

from threading import Thread, Lock

## Dataset Configuration

In [2]:
d_clinical = pd.read_csv(os.path.join('data', 'dataset_clinical_cleaned.csv'))
outputs = pd.read_csv(os.path.join('data', 'outputs_cleaned.csv'))

C = d_clinical.values

y_d = outputs["dement_fail"].values
y_c = outputs["cvd_fail"].values

In [3]:
tr_idx, ts_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=0.25).split(C, y_d))

C_ = C[tr_idx]
C_test = C[ts_idx]

y_d_ = y_d[tr_idx]
y_d_test = y_d[ts_idx]

y_c_ = y_c[tr_idx]
y_c_test = y_c[ts_idx]

In [4]:
ds_list = [C_]
ds_test = [C_test]
ds_names = ['clinic']

## Kernel Definition

In [5]:
kernel_names = ['linear', 'laplacian', 'sigmoid', 'polynomial', 'gaussian']
kernel_type = [{'linear':[1], 'laplacian':[0.2, 0.4], 'sigmoid':[1, 0.6], 'polynomial':[2, 3], 'gaussian':[0.4, 0.7]},
               {'linear':[1], 'laplacian':[0.3, 0.6], 'sigmoid':[0.8, 1.5], 'polynomial':[3, 4], 'gaussian':[0.55, 1]}]

## Other shared parameters initialization

In [6]:
estimator = ca.centeredKernelAlignment
lock_dementia = Lock()
lock_cardio = Lock()
valid_fold = 3
threads = []
pen_params = [0.5, 0.7, 0.9, 1.3]

## Thread

In [7]:
def child(sampler,estimator,ds_list,ds_test,y_d_,y_d_test,y_c_,y_c_test,lock_dementia,lock_cardio,valid_fold,exclusion_list,verbose,approach):

    #DEMENTIA
    
    result1 = sampler.sample(kernel_type, estimator, ds_list, y_d_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names)
    ut.testConfigurations(estimator, y_d_, y_d_test, w_list, ds_list, ds_test, kernel_names, lamb_list, sparsity, 'classification', lock_dementia, fileToWrite = 'results_temp/Dementia_test.txt', header = 'Dementia Linear, Laplacian, Sigmoid, Polynomial, Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)
    result1.performancesFeatures(fileToWrite = 'results_temp/Dementia_train.txt', header = '\nDementia Linear - Laplacian - Sigmoid - Polynomial - Gaussian\n' + approach + '\n', lock = lock_dementia)
        
    # CARDIO
    
    result1 = sampler.sample(kernel_type, estimator, ds_list, y_c_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names)
    ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names, lamb_list, sparsity, 'classification', lock_cardio, fileToWrite = 'results_temp/Cardio_test.txt', header = 'Cardio Linear, Laplacian, Sigmoid, Polynomial, Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)
    result1.performancesFeatures(fileToWrite = 'results_temp/Cardio_train.txt', header = '\nCardio Linear - Laplacian - Sigmoid - Polynomial - Gaussian\n' + approach + '\n', lock = lock_cardio)

## L2, Centering, Normalization

In [8]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = False, centering = True, normalizing = True)

In [9]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L2 - Centering - Normalizing'))
t.start()
threads.append(t)

## L2, Centering, K-Normalization

In [10]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = False, centering = True, normalize_kernels = True)

In [11]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L2 - Centering - K Normalizing'))
t.start()
threads.append(t)

In [12]:
for t in threads:
    t.join()

print("Operations completed")
threads = []

KeyboardInterrupt: 

## L1, Centering, Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = True, centering = True, normalizing = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L1 - Centering - Normalizing'))
t.start()
threads.append(t)

## L1, Centering, K-Normalization

In [ ]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = True, centering = True, normalize_kernels = True)

In [ ]:
t = Thread(target=child, args=(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19]], False, 'L1 - Centering - K Normalizing'))
t.start()
threads.append(t)

In [ ]:
for t in threads:
    t.join()

print("Operations completed")
threads = []